In [ ]:
import numpy as np
import pytesseract
import cv2
cap = cv2.VideoCapture("nv.avi")

if cap.isOpened() == False:
    print("카메라를 열 수 없습니다.")
    exit(1)

while(True):
    ret,img_frame = cap.read()
    if ret==False:
        print("캡쳐 실패")
        break
    key=cv2.waitKey(1)
    if key==27:
        break
    img_copy = img_frame.copy()

    # Grayscale, Gaussian blur, Otsu's threshold
    image = img_copy
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)

    # 추가
    kernel3 = np.ones((5,5),np.float32)/25
    thresh = cv2.filter2D(gray, - 1, kernel3)

    blur = cv2.GaussianBlur(thresh, (3,3), 0)
    thresh = cv2.threshold(blur, 70, 255, cv2.THRESH_BINARY_INV)[1]



    # Morph open to remove noise and invert image
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (3,3))
    opening = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, kernel, iterations=1)
    invert = 255 - opening
    bordersize = 1000
    border = cv2.copyMakeBorder(
        invert,
        top=bordersize,
        bottom=bordersize,
        left=bordersize,
        right=bordersize,
        borderType=cv2.BORDER_CONSTANT,
        value=[255]
    )



    # Perform text extraction
    data = pytesseract.image_to_string(border, lang='eng', config='—psm 10')
    if data == 'W' or data == 'w' or data == 'Ww' or data == 'VV':
       print('W')
    elif data == 'S' or data == 'S,' or data == 'S;' or data == 's' or data == 's,' or data == 's;' or data ==  '»':
       print('S')
    elif data == 'N' or data == 'Ni':
       print('N')
    elif data == 'E' or data == 'a':    #  사용자 지정 예외처리
       print('N')   
    #print(data)

    cv2.imshow('thresh', thresh)
    cv2.imshow('opening', opening)
    cv2.imshow('invert', border)
cv2.waitKey()
cv2.destroyAllWindows()